In [31]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'chess',
    'user': 'admin',  # Replace with your username
    'password': 'admin',  # Replace with your password
    'host': 'localhost',
    'port': '5433'  
}

In [13]:

# Connect to the chess database
conn = psycopg2.connect(**db_params)

# Create a cursor for executing queries
cur = conn.cursor()

# Execute a SELECT statement to retrieve data from chess_positions table
cur.execute("SELECT * FROM chess_positions;")


# Fetch and print all rows
rows = cur.fetchall()
for row in rows:
    print(row)

# Close the cursor and the connection
cur.close()
conn.close()


empty


In [ ]:
# Connect to the chess database
conn = psycopg2.connect(**db_params)

# Create a cursor for executing queries
cur = conn.cursor()

# Execute an aggregation query to count the number of occurrences of each FEN
cur.execute("""
    SELECT fen, SUM(count) as total_count
    FROM chess_positions
    GROUP BY fen;
""")


# Fetch all rows
rows = cur.fetchall()

# Close the cursor and the connection
cur.close()
conn.close()

# Create a Pandas DataFrame from the aggregated data
fen_counts_df = pd.DataFrame(rows, columns=['FEN', 'Total_Count'])

print(fen_counts_df.head())

# SAVE FEN Files in DB

In [60]:
import psycopg2
import pandas as pd
import chess
import tqdm

# Database connection parameters
db_params = {
    'dbname': 'chess',
    'user': 'admin',  # Replace with your username
    'password': 'admin',  # Replace with your password
    'host': 'localhost',
    'port': '5433'  
}

def insert_fen_into_db(fen, move_number, conn_params):
    """
    Insert a FEN into the chess_positions table in the chess database, incrementing
    the count if the same FEN with the same move_number already exists.
    """
    try:
        # Connect to the chess database
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        cur.execute("""
            INSERT INTO chess_positions (fen, move_number, count)
            VALUES (%s, %s, 1)
            ON CONFLICT (fen, move_number) 
            DO UPDATE SET count = chess_positions.count + 1;
            """, (fen, move_number))
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
    finally:
        if conn is not None:
            conn.close()

            
            
def process_games(file_path, conn_params, start_game=0, end_game=None):
    """
    Process games from a CSV file and insert FEN positions into the database.
    
    :param file_path: Path to the CSV file.
    :param conn_params: Database connection parameters.
    :param start_game: Index of the first game to process (0-indexed).
    :param end_game: Index of the last game to process (exclusive, None processes all).
    """
    # Load games from CSV
    games = pd.read_csv(file_path)
    
    # Slice the DataFrame to only include the specified range of games
    games_to_process = games.iloc[start_game:end_game]
    
    for _, game in tqdm.tqdm(games_to_process.iterrows(), total=games_to_process.shape[0]):
        # Check if 'moves' is not empty
        if not game['moves'] or pd.isna(game['moves']):
            continue  # Skip if moves are empty

        moves = eval(game['moves'])  # Convert string representation of list back to a list
        board = chess.Board()
        for move_number, move in enumerate(moves, start=1):
            try:
                board.push_san(move)
                fen = board.fen()
                # Insert FEN into the database
                insert_fen_into_db(fen, move_number, conn_params)
            except ValueError:
                # This catches invalid SAN moves and skips them
                print(f"Invalid move in game {start_game + _}: {move}")
                continue




In [ ]:
# Example usage: Process the first 100 games
process_games('../../testData/splitted_games/processed_moves.csv', db_params, start_game=100, end_game=10000)

# Example usage: Start processing from the 100th game
#process_games('../../testData/splitted_games/processed_moves.csv', db_params, start_game=100)


 59%|█████▊    | 5796/9900 [3:16:28<2:31:12,  2.21s/it]